This is where the metadata, feature space, and genres are read in and stored in the "songs" dictionary by `track_id`.

In [1]:
from pkg.spotifyclassifier import *
from pkg.graphs import *
from pkg.run import *
from pkg.wrangling import *
from pkg.constants import LABEL_LIST

In [2]:
default_song_data = FrozenMap(read_data('data/data.csv'))
print('Song data reading complete.')


Song data reading complete.


In [3]:
# genres = FrozenMap(read_genres('data/data.csv'))
# print('Genre data reading complete.')


Genre data reading complete.


In [4]:
default_feature_names = tuple(next(iter(default_song_data.values()))['features'].keys())

In [5]:
# Sanity check
l, dl = 0, 0
for x in default_song_data.values():
    if x['label'] == 1: l += 1
    else: dl += 1
print(' likes:', l, '\n', 'dislikes:', dl)

 likes: 1919 
 dislikes: 3789


In [6]:
test_cluster_size(default_song_data, 10)

2 2584.0730140650658
3 2183.5162410295115
4 1889.928671274101
5 1764.8594314268544
6 1646.592423018276
7 1562.6647413388932
8 1484.5573009010418
9 1426.6460237649508
10 1382.9505083701033


In [7]:
clustered_song_data, songs_by_cluster = {}, {}
clustered_song_data, songs_by_cluster = get_kmeans_clusters(default_song_data, NUM_CLUSTERS)

In [8]:
if set(next(iter(default_song_data.values()))['features'].keys()) == set(next(iter(clustered_song_data.values()))['features'].keys()):
    raise ValueError('Default features messed up.')

In [9]:
default_training_data, default_validation_data, clustered_training_data, clustered_validation_data = get_experiment_split(default_song_data, clustered_song_data)
training_clusters = get_train_clusters(clustered_training_data, songs_by_cluster[NUM_CLUSTERS])

In [12]:
print('----\nUnclustered\n----')
active_unclustered_results = run_active_suite(default_song_data, default_training_data, default_validation_data, SUPPORTED_ALGS, AL_STRATS)

----
Unclustered
----
svc accs w/ random
65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	88.16	66.17	87.81	67.04	86.24	35.05	87.81	74.14	67.39	65.9	72.12	83.17	74.32	83.08	81.24	79.92	76.95	72.56	72.56	72.56	73.44	77.56	76.95	76.86	77.73	77.38	77.38	76.51	75.98	75.81	74.14	74.14	74.23	72.65	72.65	81.68	82.38	85.36	85.36	85.36	84.92	84.57	85.01	85.01	85.1	85.1	85.01	81.5	81.15	80.98	80.98	80.98	79.92	80.1	84.92	85.62	86.15	86.76	86.76	86.76	86.76	86.76	86.5	86.5	86.41	86.41	86.5	86.06	86.76	86.76	87.2	87.37	87.37	87.2	87.2	87.2	87.2	87.2	87.46	87.37	87.64	87.64	87.64	87.64	87.46	87.55	87.55	87.55	87.55	87.55	87.55	87.55	
svc accs w/ uncertainty
65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	66.95	66.78	65.81	65.81	66.17	77.12	77.12	77.21	77.21	

/Users/adi/repos/spotify-music-discovery/venv/lib/python3.5/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


lsvc accs w/ random
34.35	34.35	34.35	86.06	85.18	85.88	88.43	88.43	86.24	56.26	56.26	68.88	82.55	81.77	71.86	83.08	87.11	88.43	88.95	88.95	88.43	89.74	89.65	89.57	89.57	89.04	89.74	89.3	89.3	89.21	89.21	88.95	88.95	88.95	89.21	89.13	89.13	89.13	89.13	89.13	89.21	89.21	88.86	88.86	88.86	89.04	88.86	89.04	89.04	89.04	89.13	89.13	89.13	89.13	89.13	89.13	89.21	89.21	89.21	89.21	89.21	89.21	89.21	89.21	88.95	89.21	89.21	89.04	89.04	89.04	89.04	88.95	88.86	88.95	88.95	88.95	88.95	88.95	88.95	88.86	88.78	88.95	88.95	88.95	89.13	89.13	89.13	89.13	89.3	89.21	89.21	89.21	89.13	89.13	89.13	89.13	89.13	89.13	89.13	89.13	89.13	
lsvc accs w/ uncertainty
76.77	73.97	67.22	85.97	85.97	87.29	87.29	87.29	87.11	87.11	86.15	87.29	87.64	87.64	87.64	87.73	87.55	87.64	87.64	87.64	87.73	87.73	87.73	87.64	87.64	87.64	87.64	87.55	87.73	87.55	87.55	88.51	88.43	88.43	88.43	88.43	88.43	88.43	88.43	88.43	88.43	88.43	88.34	88.34	88.51	88.51	88.51	88.51	88.43	88.51	88.51	88.51	88.51	88.51	88.51	88.51	88.51	88.43	88.

In [13]:
print('----\nClustered\n----')
active_clustered_results = run_active_suite(default_song_data, clustered_training_data, clustered_validation_data, SUPPORTED_ALGS, AL_STRATS)

----
Clustered
----
svc accs w/ random
65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	66.25	68.62	68.62	66.25	67.83	67.83	67.39	75.72	72.56	72.48	71.07	68.44	68.44	68.36	67.13	73.61	76.24	76.24	76.24	75.81	75.81	74.93	81.41	81.41	81.41	80.71	81.06	80.54	84.83	85.71	85.8	85.8	85.71	85.71	86.5	86.5	86.5	86.94	86.94	87.02	87.02	86.94	86.85	86.85	86.85	87.37	87.2	87.11	87.11	86.76	86.76	87.2	87.2	87.2	87.64	87.64	87.55	87.55	87.81	87.73	87.73	87.64	87.73	87.81	87.81	87.9	87.9	87.9	87.73	87.73	87.73	87.73	87.64	87.81	87.81	87.99	87.99	87.99	87.99	88.16	
svc accs w/ uncertainty
65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	65.64	66.52	66.6	66.6	65.81	72.56	71.51	81.77	81.41	84.66	84.83	84.57	83.34	83.34	83.34	85.71	86.24	86.24	86.59	86.59	86.32	86.67	86.67	87.2	87.64	87.64	87.99	87.99	87.99	87.99	88.

In [14]:
print('\n----\nClustered w/ Cluster Sampling\n----')
active_cluster_sampled_results = run_clusters_suite(default_song_data, clustered_training_data, clustered_validation_data, training_clusters)


----
Clustered w/ Cluster Sampling
----
sgd accs w/ random
65.64	65.64	54.77	70.46	56.26	67.39	69.41	86.59	86.59	86.59	86.59	86.59	86.59	86.59	86.59	82.2	89.57	34.35	89.04	89.04	89.04	88.25	88.25	86.15	86.5	86.5	
sgd accs w/ uncertainty
67.13	65.64	75.37	34.35	73.61	77.12	81.06	81.06	88.51	89.74	85.45	86.76	86.76	87.2	87.2	85.88	89.57	89.57	86.15	86.15	86.15	86.15	86.5	86.5	86.5	83.34	


In [10]:
best_classifier = get_highest_benchmark(
    default_song_data,
    default_training_data,
    default_validation_data,
    clustered_training_data,
    clustered_validation_data
)

  (0, 0)	0.447
  (0, 1)	0.003931894276853357
  (0, 2)	0.68
  (0, 3)	0.631
  (0, 4)	0.09124870637748937
  (0, 5)	0.381
  (0, 6)	0.000171
  (0, 7)	0.6363636363636364
  (0, 8)	0.0902
  (0, 9)	0.7653473954789445
  (0, 10)	0.04
  (0, 11)	0.5013128928732462
  (0, 12)	0.75
  (0, 13)	0.36
  (0, 0)	0.812
  (0, 1)	7.499980000053333e-06
  (0, 2)	0.24
  (0, 3)	0.623
  (0, 4)	0.21647972034359952
  (0, 5)	0.149
  (0, 6)	0.0
  (0, 7)	0.7272727272727273
  (0, 8)	0.228
  (0, 9)	0.6278256596355938
  (0, 10)	0.954
  (0, 11)	0.24949738747579775
  (0, 12)	0.5
  (0, 13)	0.381
  (0, 0)	0.635
  (0, 1)	0.0001388805820327336
  (0, 2)	0.41000000000000003
  (0, 3)	0.503
  (0, 4)	0.08395039173507503
  (0, 5)	0.56
  (0, 6)	9.5e-05
  (0, 7)	0.4545454545454546
  (0, 8)	0.0865
  (0, 9)	0.8267369773947228
  (0, 10)	0.0297
  (0, 11)	0.6284009230034746
  (0, 12)	0.75
  (0, 13)	0.271
  (0, 0)	0.353
  (0, 1)	1.3309488317554962e-05
  (0, 2)	0.4
  (0, 3)	0.721
  (0, 4)	0.062439217844359816
  (0, 5)	0.668
  (0, 6)	0.0
  (0, 7

  (0, 0)	0.611
  (0, 1)	0.00047102255346366696
  (0, 2)	0.43
  (0, 3)	0.686
  (0, 4)	0.1385323497531952
  (0, 5)	0.467
  (0, 6)	0.904
  (0, 7)	0.7272727272727273
  (0, 8)	0.0676
  (0, 9)	0.7773115596885158
  (0, 10)	0.0759
  (0, 11)	0.5490756703710581
  (0, 12)	0.75
  (0, 13)	0.437
  (0, 0)	0.0813
  (0, 1)	0.05651244453824313
  (0, 2)	0.75
  (0, 3)	0.631
  (0, 4)	0.0440360799533906
  (0, 5)	0.779
  (0, 6)	0.0
  (0, 7)	0.09090909090909091
  (0, 8)	0.263
  (0, 9)	0.8896197172450291
  (0, 10)	0.0469
  (0, 11)	0.3739702410948731
  (0, 12)	0.75
  (0, 13)	0.684
  (0, 0)	0.0608
  (0, 1)	0.0013155441109299899
  (0, 2)	0.47000000000000003
  (0, 3)	0.781
  (0, 4)	0.11917396423864816
  (0, 5)	0.824
  (0, 6)	0.802
  (0, 7)	0.6363636363636364
  (0, 8)	0.072
  (0, 9)	0.8207076434565661
  (0, 10)	0.0574
  (0, 11)	0.39497652706681174
  (0, 12)	0.75
  (0, 13)	0.708
  (0, 0)	0.465
  (0, 1)	0.036389593437274645
  (0, 2)	0.73
  (0, 3)	0.76
  (0, 4)	0.04955423634218697
  (0, 5)	0.672
  (0, 6)	0.0
  (0, 7)	

  (0, 0)	0.705
  (0, 1)	0.0018607331332830732
  (0, 2)	0.63
  (0, 3)	0.283
  (0, 4)	0.06157030455410823
  (0, 5)	0.246
  (0, 6)	4.42e-05
  (0, 7)	0.4545454545454546
  (0, 8)	0.111
  (0, 9)	0.7806758902245406
  (0, 10)	0.0367
  (0, 11)	0.5095827918202795
  (0, 12)	0.75
  (0, 13)	0.078
  (0, 0)	0.005
  (0, 1)	4.619035301620148e-05
  (0, 2)	0.44
  (0, 3)	0.528
  (0, 4)	0.06326190049160442
  (0, 5)	0.762
  (0, 6)	0.0
  (0, 7)	0.5454545454545454
  (0, 8)	0.0849
  (0, 9)	0.8848567324412187
  (0, 10)	0.0371
  (0, 11)	0.5062355780707106
  (0, 12)	0.75
  (0, 13)	0.41
  (0, 0)	0.0246
  (0, 1)	0.0014034962573433137
  (0, 2)	0.58
  (0, 3)	0.483
  (0, 4)	0.050322611908790844
  (0, 5)	0.721
  (0, 6)	0.0
  (0, 7)	0.0
  (0, 8)	0.14
  (0, 9)	0.9035306569894911
  (0, 10)	0.0334
  (0, 11)	0.24610773678487124
  (0, 12)	0.75
  (0, 13)	0.383
  (0, 0)	0.0231
  (0, 1)	0.0006173078776551834
  (0, 2)	0.37
  (0, 3)	0.552
  (0, 4)	0.08152300447935942
  (0, 5)	0.771
  (0, 6)	0.852
  (0, 7)	0.18181818181818182
  (0

  (0, 0)	0.0208
  (0, 1)	3.0976107873236144e-05
  (0, 2)	0.32
  (0, 3)	0.71
  (0, 4)	0.06829769860517138
  (0, 5)	0.61
  (0, 6)	0.429
  (0, 7)	0.18181818181818182
  (0, 8)	0.232
  (0, 9)	0.8740077114992061
  (0, 10)	0.0756
  (0, 11)	0.5760814789274064
  (0, 12)	0.75
  (0, 13)	0.298
  (0, 0)	0.33
  (0, 1)	0.12311288598468499
  (0, 2)	0.87
  (0, 3)	0.653
  (0, 4)	0.061896982531500895
  (0, 5)	0.71
  (0, 6)	0.0
  (0, 7)	0.7272727272727273
  (0, 8)	0.143
  (0, 9)	0.8289294624631435
  (0, 10)	0.0554
  (0, 11)	0.48028538843062885
  (0, 12)	0.75
  (0, 13)	0.409
  (0, 0)	0.27
  (0, 1)	0.09183046940446254
  (0, 2)	0.77
  (0, 3)	0.515
  (0, 4)	0.056446222622268914
  (0, 5)	0.904
  (0, 6)	0.0
  (0, 7)	0.6363636363636364
  (0, 8)	0.28
  (0, 9)	0.8731760792318741
  (0, 10)	0.302
  (0, 11)	0.22998169906903965
  (0, 12)	0.75
  (0, 13)	0.751
  (0, 0)	0.763
  (0, 1)	0.020328707694874718
  (0, 2)	0.6900000000000001
  (0, 3)	0.561
  (0, 4)	0.08677992988538144
  (0, 5)	0.565
  (0, 6)	0.948
  (0, 7)	0.0909

  (0, 0)	0.00428
  (0, 1)	0.00015173769060425362
  (0, 2)	0.45
  (0, 3)	0.681
  (0, 4)	0.05019394590234975
  (0, 5)	0.768
  (0, 6)	0.000127
  (0, 7)	0.09090909090909091
  (0, 8)	0.224
  (0, 9)	0.9034739547894457
  (0, 10)	0.064
  (0, 11)	0.5277192796329204
  (0, 12)	0.75
  (0, 13)	0.304
  (0, 0)	0.0279
  (0, 1)	0.0010952589840712806
  (0, 2)	0.48
  (0, 3)	0.562
  (0, 4)	0.12108863695354545
  (0, 5)	0.784
  (0, 6)	0.405
  (0, 7)	0.9090909090909092
  (0, 8)	0.144
  (0, 9)	0.8098586225145535
  (0, 10)	0.0431
  (0, 11)	0.42160040315094294
  (0, 12)	0.75
  (0, 13)	0.684
  (0, 0)	0.0738
  (0, 1)	0.001194425386294208
  (0, 2)	0.5
  (0, 3)	0.826
  (0, 4)	0.05199749797532064
  (0, 5)	0.899
  (0, 6)	0.86
  (0, 7)	0.36363636363636365
  (0, 8)	0.0657
  (0, 9)	0.8682051863612308
  (0, 10)	0.0386
  (0, 11)	0.43233694931437816
  (0, 12)	0.75
  (0, 13)	0.378
  (0, 0)	0.176
  (0, 1)	0.0016160195001384758
  (0, 2)	0.77
  (0, 3)	0.694
  (0, 4)	0.04699929707142802
  (0, 5)	0.77
  (0, 6)	1.1e-05
  (0, 7)	0

...

  (0, 0)	4.01e-05
  (0, 1)	0.0004270226707966874
  (0, 2)	0.39
  (0, 3)	0.719
  (0, 4)	0.09696682422218338
  (0, 5)	0.79
  (0, 6)	0.798
  (0, 7)	0.7272727272727273
  (0, 8)	0.108
  (0, 9)	0.8268881832615105
  (0, 10)	0.0395
  (0, 11)	0.5170835212052092
  (0, 12)	0.75
  (0, 13)	0.219
  (0, 0)	0.258
  (0, 1)	0.0002140946671761256
  (0, 2)	0.63
  (0, 3)	0.665
  (0, 4)	0.0455809075242321
  (0, 5)	0.748
  (0, 6)	0.0
  (0, 7)	0.8181818181818182
  (0, 8)	0.123
  (0, 9)	0.8831745671732063
  (0, 10)	0.0377
  (0, 11)	0.46961249767922975
  (0, 12)	0.75
  (0, 13)	0.708
  (0, 0)	0.0378
  (0, 1)	0.0015432577894077997
  (0, 2)	0.59
  (0, 3)	0.71
  (0, 4)	0.04650914085641431
  (0, 5)	0.627
  (0, 6)	0.0
  (0, 7)	1.0
  (0, 8)	0.144
  (0, 9)	0.8522718681484841
  (0, 10)	0.145
  (0, 11)	0.6020741055088454
  (0, 12)	0.75
  (0, 13)	0.626
  (0, 0)	0.623
  (0, 1)	0.3412557328418553
  (0, 2)	0.9
  (0, 3)	0.375
  (0, 4)	0.06496797831727144
  (0, 5)	0.408
  (0, 6)	0.0
  (0, 7)	0.6363636363636364
  (0, 8)	0.123
 

  (0, 13)	0.112
  (0, 0)	0.00103
  (0, 1)	0.014953102982201571
  (0, 2)	0.53
  (0, 3)	0.667
  (0, 4)	0.056977039523300806
  (0, 5)	0.813
  (0, 6)	0.0
  (0, 7)	0.8181818181818182
  (0, 8)	0.0861
  (0, 9)	0.9061767596582746
  (0, 10)	0.0553
  (0, 11)	0.3950507917141872
  (0, 12)	0.75
  (0, 13)	0.636
  (0, 0)	0.0448
  (0, 1)	0.1404054589187762
  (0, 2)	0.84
  (0, 3)	0.799
  (0, 4)	0.05559931066212307
  (0, 5)	0.649
  (0, 6)	0.0
  (0, 7)	1.0
  (0, 8)	0.109
  (0, 9)	0.8835903833068722
  (0, 10)	0.274
  (0, 11)	0.5115083680343739
  (0, 12)	0.75
  (0, 13)	0.232
  (0, 0)	0.0061
  (0, 1)	0.001561876787376186
  (0, 2)	0.71
  (0, 3)	0.867
  (0, 4)	0.05981047664349936
  (0, 5)	0.811
  (0, 6)	0.0243
  (0, 7)	0.7272727272727273
  (0, 8)	0.0648
  (0, 9)	0.933091403946473
  (0, 10)	0.062
  (0, 11)	0.5495371721083203
  (0, 12)	0.75
  (0, 13)	0.687
  (0, 0)	0.331
  (0, 1)	0.036389593437274645
  (0, 2)	0.73
  (0, 3)	0.806
  (0, 4)	0.05398625111816887
  (0, 5)	0.707
  (0, 6)	0.0
  (0, 7)	0.636363636363636

  (0, 13)	0.379
  (0, 0)	0.86
  (0, 1)	0.00790178845237365
  (0, 2)	0.6
  (0, 3)	0.663
  (0, 4)	0.033661199568216935
  (0, 5)	0.219
  (0, 6)	0.0
  (0, 7)	0.36363636363636365
  (0, 8)	0.318
  (0, 9)	0.8159824601194525
  (0, 10)	0.0306
  (0, 11)	0.4753520939978252
  (0, 12)	0.75
  (0, 13)	0.569
  (0, 0)	0.261
  (0, 1)	0.06415904319493243
  (0, 2)	0.75
  (0, 3)	0.392
  (0, 4)	0.04757105315632753
  (0, 5)	0.852
  (0, 6)	0.0
  (0, 7)	0.0
  (0, 8)	0.0752
  (0, 9)	0.912810917063582
  (0, 10)	0.182
  (0, 11)	0.522149431079755
  (0, 12)	0.75
  (0, 13)	0.325
  (0, 0)	0.0131
  (0, 1)	0.0004968081989876599
  (0, 2)	0.5700000000000001
  (0, 3)	0.888
  (0, 4)	0.00999027485509757
  (0, 5)	0.675
  (0, 6)	0.00176
  (0, 7)	1.0
  (0, 8)	0.184
  (0, 9)	0.8659938005594616
  (0, 10)	0.0521
  (0, 11)	0.5227276344057502
  (0, 12)	1.0
  (0, 13)	0.605
  (0, 0)	0.208
  (0, 1)	0.0974591924897724
  (0, 2)	0.86
  (0, 3)	0.748
  (0, 4)	0.04704719870153163
  (0, 5)	0.652
  (0, 6)	0.0
  (0, 7)	0.6363636363636364
  (0,

  (0, 13)	0.655
  (0, 0)	0.133
  (0, 1)	0.12520876134806497
  (0, 2)	0.81
  (0, 3)	0.402
  (0, 4)	0.05322650898489762
  (0, 5)	0.794
  (0, 6)	0.0
  (0, 7)	1.0
  (0, 8)	0.133
  (0, 9)	0.9132645346639449
  (0, 10)	0.125
  (0, 11)	0.8675967429647509
  (0, 12)	0.5
  (0, 13)	0.702
  (0, 0)	0.0912
  (0, 1)	0.0030033967528467543
  (0, 2)	0.72
  (0, 3)	0.528
  (0, 4)	0.05347910653433934
  (0, 5)	0.904
  (0, 6)	0.0
  (0, 7)	0.0
  (0, 8)	0.0424
  (0, 9)	0.8951765328494745
  (0, 10)	0.0387
  (0, 11)	0.7349282550460177
  (0, 12)	0.75
  (0, 13)	0.757
  (0, 0)	0.165
  (0, 1)	0.005757722741310785
  (0, 2)	0.5700000000000001
  (0, 3)	0.684
  (0, 4)	0.04256561130834948
  (0, 5)	0.602
  (0, 6)	0.0
  (0, 7)	0.6363636363636364
  (0, 8)	0.351
  (0, 9)	0.8008240719739925
  (0, 10)	0.0258
  (0, 11)	0.3307216932339602
  (0, 12)	0.75
  (0, 13)	0.684
  (0, 0)	0.00278
  (0, 1)	0.04411016808717082
  (0, 2)	0.86
  (0, 3)	0.493
  (0, 4)	0.04275861031801113
  (0, 5)	0.833
  (0, 6)	0.0
  (0, 7)	0.18181818181818182
  

  (0, 0)	0.0425
  (0, 1)	0.00013249964666760887
  (0, 2)	0.27
  (0, 3)	0.803
  (0, 4)	0.05735663209208699
  (0, 5)	0.989
  (0, 6)	0.913
  (0, 7)	0.7272727272727273
  (0, 8)	0.0933
  (0, 9)	0.819497996522265
  (0, 10)	0.067
  (0, 11)	0.5015516006683819
  (0, 12)	0.75
  (0, 13)	0.232
  (0, 0)	0.0169
  (0, 1)	0.09824707134114309
  (0, 2)	0.76
  (0, 3)	0.624
  (0, 4)	0.07324493440261655
  (0, 5)	0.647
  (0, 6)	0.0038
  (0, 7)	0.2727272727272727
  (0, 8)	0.221
  (0, 9)	0.900071822786724
  (0, 10)	0.0335
  (0, 11)	0.5544333342174362
  (0, 12)	0.75
  (0, 13)	0.379
  (0, 0)	0.535
  (0, 1)	7.499980000053333e-06
  (0, 2)	0.24
  (0, 3)	0.726
  (0, 4)	0.03555749142505122
  (0, 5)	0.135
  (0, 6)	0.0
  (0, 7)	0.8181818181818182
  (0, 8)	0.337
  (0, 9)	0.561710894382702
  (0, 10)	0.954
  (0, 11)	0.25239370872344374
  (0, 12)	0.5
  (0, 13)	0.538
  (0, 0)	0.00522
  (0, 1)	0.00034421336781197153
  (0, 2)	0.56
  (0, 3)	0.312
  (0, 4)	0.07721937721196918
  (0, 5)	0.705
  (0, 6)	0.0
  (0, 7)	0.090909090909